In [1]:
from hyprland.hyprland_monitors import get_gdk_monitor_id_from_name, get_hyprctl_monitor_id_from_name, get_all_monitors

In [5]:
get_all_monitors()

{0: 'eDP-1', 1: 'HDMI-A-1'}

# Gtk 

## Icons

In [2]:
import gi
gi.require_version('Gtk', '3.0')
from gi.repository import Gtk, GdkPixbuf, Gio

class IconListRow(Gtk.ListBoxRow):
    def __init__(self, name):
        super().__init__()
        self.connect("focus-in-event", update_info, name)
        self.connect("activate", on_row_activate, name)

        eb = Gtk.EventBox.new()
        self.add(eb)
        eb.connect("button-press-event", update_info, name)
        box = Gtk.Box.new(Gtk.Orientation.HORIZONTAL, 0)
        eb.add(box)

        pixbuf = gtk_icon_theme.load_icon(name, 24,
                                          Gtk.IconLookupFlags.FORCE_SIZE |
                                          Gtk.IconLookupFlags.GENERIC_FALLBACK |
                                          Gtk.IconLookupFlags.USE_BUILTIN)
        img = Gtk.Image.new_from_pixbuf(pixbuf)
        box.pack_start(img, False, False, 6)

        lbl = Gtk.Label.new(name)
        box.pack_start(lbl, False, False, 0)


def update_info(ebox, ebtn, name):
    global icon_info
    icon_info.update(name)

In [ ]:
import gi
gi.require_version("Gtk", "3.0")
from gi.repository import Gtk
from gi.repository.GdkPixbuf import Pixbuf

icons = ["battery-060", "edit-paste", "edit-copy", "nm-signal-50"]
class IconViewWindow(Gtk.Window):
    def __init__(self):
        super().__init__()
        self.set_default_size(200, 200)
        liststore = Gtk.ListStore(Pixbuf, str)
        iconview = Gtk.IconView.new()
        iconview.set_model(liststore)
        iconview.set_pixbuf_column(0)
        iconview.set_text_column(1)
        for icon in icons:
            pixbuf = Gtk.IconTheme.get_default().load_icon(icon, 64, 0)
            liststore.append([pixbuf, icon])
        self.add(iconview)

win = IconViewWindow()
win.connect("destroy", Gtk.main_quit)
win.show_all()
Gtk.main()

In [32]:
theme = Gtk.IconTheme.get_default()
theme

<Gtk.IconTheme object at 0x7092ef27fe80 (GtkIconTheme at 0x6112d5002f00)>

# Fabric

In [10]:
import psutil, os, signal
for proc in psutil.process_iter(['pid', 'name', 'cmdline']):
    if "python" in proc.info['name'] and "fabric_bar.py" in " ".join(proc.info['cmdline']):
        print(proc.info)
        # os.kill(proc.info['pid'], signal.SIGTERM)

In [1]:
from fabric.hyprland.service import Hyprland, HyprlandEvent
import json

In [3]:
import fabric, os
os.path.dirname(fabric.__file__)

'/usr/lib/python3.13/site-packages/fabric'

In [2]:
connection: Hyprland | None = None


def get_hyprland_connection() -> Hyprland:
    global connection
    if not connection:
        connection = Hyprland()

    return connection

In [3]:
connection = get_hyprland_connection()

/usr/lib/python3.13/site-packages/fabric/hyprland/service.py:74: Warning: g_object_notify: object class 'fabric+hyprland+service+Hyprland' has no property named 'ready'
  return self.notify("ready")


In [4]:
connection.send_command("j/workspaces")

HyprlandReply(command='j/workspaces', reply=b'[{\n    "id": 1,\n    "name": "1",\n    "monitor": "eDP-1",\n    "monitorID": 0,\n    "windows": 0,\n    "hasfullscreen": false,\n    "lastwindow": "0x0",\n    "lastwindowtitle": ""\n},{\n    "id": 2,\n    "name": "2",\n    "monitor": "eDP-1",\n    "monitorID": 0,\n    "windows": 0,\n    "hasfullscreen": false,\n    "lastwindow": "0x0",\n    "lastwindowtitle": ""\n},{\n    "id": 3,\n    "name": "3",\n    "monitor": "eDP-1",\n    "monitorID": 0,\n    "windows": 0,\n    "hasfullscreen": false,\n    "lastwindow": "0x0",\n    "lastwindowtitle": ""\n},{\n    "id": 4,\n    "name": "4",\n    "monitor": "eDP-1",\n    "monitorID": 0,\n    "windows": 0,\n    "hasfullscreen": false,\n    "lastwindow": "0x0",\n    "lastwindowtitle": ""\n},{\n    "id": 5,\n    "name": "5",\n    "monitor": "eDP-1",\n    "monitorID": 0,\n    "windows": 0,\n    "hasfullscreen": false,\n    "lastwindow": "0x0",\n    "lastwindowtitle": ""\n},{\n    "id": 6,\n    "name": "6",

In [9]:
tuple(
            workspace["id"]
            for workspace in json.loads(
                str(connection.send_command("j/workspaces").reply.decode())
            )
            if workspace["monitor"]=="HDMI-A-1"
        )

(14, 13, 12)

In [20]:
workspaces=json.loads(
        str(connection.send_command("j/workspaces").reply.decode())
    )

In [46]:
clients=json.loads(
        str(connection.send_command("j/clients").reply.decode())
    )
clients[0]

{'address': '0x5f0b1ff14860',
 'mapped': True,
 'hidden': False,
 'at': [6, 6],
 'size': [1908, 1068],
 'workspace': {'id': 12, 'name': '12'},
 'floating': False,
 'pseudo': False,
 'monitor': 1,
 'class': 'org.kde.kate',
 'title': 'keybinds.conf - custom  — Kate',
 'initialClass': 'org.kde.kate',
 'initialTitle': 'Kate',
 'pid': 108926,
 'xwayland': False,
 'pinned': False,
 'fullscreen': 0,
 'fullscreenClient': 0,
 'grouped': [],
 'tags': [],
 'swallowing': '0x0',
 'focusHistoryID': 3,
 'inhibitingIdle': False}

In [47]:
def hyprland_monitor_info(monitor_name:str):
    monitors = json.loads(
        str(connection.send_command("j/monitors").reply.decode())
    )
    for monior in monitors:
        if monior["name"]==monitor_name:
            return monior
    raise ValueError(f"Monitor {monitor_name} not found")

def active_workspace_id_on_monitor(monitor_name:str):
    monitor = hyprland_monitor_info(monitor_name)
    return monitor['activeWorkspace']['id']

def workspace_ids_on_monitor(monitor_name:str):
    workspaces=json.loads(
        str(connection.send_command("j/workspaces").reply.decode())
    )
    return [workspace["id"] for workspace in workspaces if workspace["monitor"]==monitor_name]

def active_client_on_monitor(monitor_name:str):
    active_workspace_id = active_workspace_id_on_monitor(monitor_name)
    workspaces=json.loads(
        str(get_hyprland_connection().send_command("j/workspaces").reply.decode())
    )
    active_workspace_on_monitor = [workspace for workspace in workspaces if workspace["id"]==active_workspace_id][0]
    clients=json.loads(
        str(get_hyprland_connection().send_command("j/clients").reply.decode())
    )
    return [client for client in clients if client["address"]==active_workspace_on_monitor["lastwindow"]][0]

In [50]:
active_client_on_monitor("eDP-1")

{'address': '0x5f0b20a83110',
 'mapped': True,
 'hidden': False,
 'at': [1926, 6],
 'size': [1908, 1068],
 'workspace': {'id': 1, 'name': '1'},
 'floating': False,
 'pseudo': False,
 'monitor': 0,
 'class': 'kitty',
 'title': '~/Downloads',
 'initialClass': 'kitty',
 'initialTitle': 'kitty',
 'pid': 127515,
 'xwayland': False,
 'pinned': False,
 'fullscreen': 0,
 'fullscreenClient': 0,
 'grouped': [],
 'tags': [],
 'swallowing': '0x0',
 'focusHistoryID': 1,
 'inhibitingIdle': False}

In [48]:
active_workspace_id_on_monitor("eDP-1")

1

# Basic windows

In [1]:
import fabric # importing the base pacakge
from fabric import Application # prepare the application class which manages multi-config setups
from fabric.widgets.box import Box # gets the Box class
from fabric.widgets.label import Label # gets the Label class
from fabric.widgets.window import Window # grabs the Window class from Fabric


if __name__ == "__main__":
    box_1 = Box(
        orientation="v", # vertical
        children=Label(label="this is the first box")
    )
    box_2 = Box(
        spacing=28, # adds some spacing between the children
        orientation="h", # horizontal
        children=[
            Label(label="this is the first element in the second box"),
            Label(label="btw, this box elements will get added horizontally")
        ]
    )
    box_1.add(box_2) # append box_2 inside box_1 along with the label already in there

    window = Window(child=box_1) # there's no need showing this window using `show_all()`; it'll show them itself because the children are already passed
    app = Application("default", window) # define a new config named "defualt" which holds `window`

    app.run() # run the event loop (run the config)

2025-01-26 18:56:46.200 | DEBUG    | fabric.core.application:do_activate:311 - [Fabric] opening a DBus client for Application with name default
